In [3]:
from pathlib import Path

embed_dir = Path('/Users/mma0448/tmp/embeds')

In [4]:
from huggingface_hub import from_pretrained_keras
import tensorflow as tf

2025-01-29 16:02:43.018983: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Authenticate user for HuggingFace if needed. Enter token below if requested.
from huggingface_hub.utils import HfFolder
from huggingface_hub import notebook_login

if HfFolder.get_token() is None:
    from huggingface_hub import notebook_login
    notebook_login()

In [6]:
loaded_model = from_pretrained_keras("google/derm-foundation")

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
from PIL import Image
from io import BytesIO
from IPython.display import Image as IPImage, display

In [13]:
# from pathlib import Path
# import datetime
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [14]:
data_root = Path('/Users/mma0448/tmp/image_data')

In [15]:
class ImageForDFDataset(Dataset):
    
    def __init__(
        self, 
        metadata_path: Path, 
        transform=None, 
    ):
        self.data_list = pd.read_csv(metadata_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(
        self, 
        idx: int,
    ):
        row = self.data_list.iloc[idx]
        img_path, label = row['filename'], row['label']

        # keep either Albumentations or torchvision happy
        img = Image.open(img_path)

        
        if self.transform is not None:
            img = self.transform(img)
            
        return {
            'image': img, 
            'label': label,
            'filename': str(img_path)
        }

In [16]:
from torchvision.transforms import Compose

class ConvertToMakeModelHappy(object):
    def __init__(self, format='PNG'):
        self.format = format

    def __call__(self, sample):
        buf = BytesIO()
        sample.convert('RGB').save(buf, self.format)
        image_bytes = buf.getvalue()
        # return image_bytes
        input_tensor= tf.train.Example(features=tf.train.Features(
            feature={
                'image/encoded': tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[image_bytes]))
            })
        ).SerializeToString()
        return input_tensor        

In [17]:
transforms = ConvertToMakeModelHappy()

In [18]:
# init dataset and loader
train_set = ImageForDFDataset(
    data_root / 'train.csv', 
    transform=transforms
)

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)

In [20]:
from tqdm import tqdm
import pandas as pd
import numpy as np

infer = loaded_model.signatures["serving_default"]

df = pd.DataFrame(columns=['feature_path', 'label'])
counter = 0
for batch in tqdm(train_loader):
    for img, label in zip(batch['image'], batch['label']):
        counter += 1
        
        save_path = embed_dir / f'{counter:06d}.npy'
        if save_path.exists():
            continue
        
        output = infer(inputs=tf.constant([img]))
        
        # Extract the embedding vector
        embedding_vector = output['embedding'].numpy().flatten()
        
        np.save(save_path, embedding_vector)
        df.loc[counter, 'feature_path'] = save_path
        df.loc[counter, 'label'] = label

len(df)

100%|██████████████████████████████████████████████████████████| 706/706 [10:17:12<00:00, 52.45s/it]


9841

In [24]:
md_dir = Path('/Users/mma0448/tmp/dfound_md')
md_dir.mkdir()

In [25]:
df.to_csv(md_dir / 'train_small.csv')

In [27]:
# init dataset and loader
test_set = ImageForDFDataset(
    data_root / 'test.csv', 
    transform=transforms
)

test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

In [28]:
embed_dir = Path('/Users/mma0448/tmp/test_embeds')

In [29]:
embed_dir.mkdir()

In [30]:
infer = loaded_model.signatures["serving_default"]

df = pd.DataFrame(columns=['feature_path', 'label'])
counter = 0
for batch in tqdm(test_loader):
    for img, label in zip(batch['image'], batch['label']):
        counter += 1
        
        save_path = embed_dir / f'test_{counter:06d}.npy'
        if save_path.exists():
            continue
        
        output = infer(inputs=tf.constant([img]))
        
        # Extract the embedding vector
        embedding_vector = output['embedding'].numpy().flatten()
        
        np.save(save_path, embedding_vector)
        df.loc[counter, 'feature_path'] = save_path
        df.loc[counter, 'label'] = label

len(df)

100%|███████████████████████████████████████████████████████████| 125/125 [2:16:14<00:00, 65.39s/it]


2000

In [32]:
df.to_csv(md_dir / 'test.csv', index=False)